In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input, Dense, Flatten, Reshape
from tensorflow.keras.models import Model

In [ ]:
x_train = np.load('Lab CIA 2/Lab CIA 2/mnist.npz')['x_train']
x_test = np.load('Lab CIA 2/Lab CIA 2/mnist.npz')['x_test']
y_train = np.load('Lab CIA 2/Lab CIA 2/mnist.npz')['y_train']
y_test = np.load('Lab CIA 2/Lab CIA 2/mnist.npz')['y_test']

In [ ]:
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

x_train = np.reshape(x_train, (len(x_train), 28, 28, 1))
x_test = np.reshape(x_test, (len(x_test), 28, 28, 1))

In [ ]:
noise_factor = 0.5
x_train_noisy = x_train + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_train.shape)
x_test_noisy = x_test + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_test.shape)


x_train_noisy = np.clip(x_train_noisy, 0., 1.)
x_test_noisy = np.clip(x_test_noisy, 0., 1.)


input_img = Input(shape=(28, 28, 1))

In [ ]:
x = Flatten()(input_img)
encoded = Dense(128, activation='relu')(x)
encoded = Dense(64, activation='relu')(encoded)
encoded = Dense(32, activation='relu')(encoded)


decoded = Dense(64, activation='relu')(encoded)
decoded = Dense(128, activation='relu')(decoded)
decoded = Dense(28 * 28, activation='sigmoid')(decoded)
decoded = Reshape((28, 28, 1))(decoded)

autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

In [ ]:
autoencoder.fit(x_train_noisy, x_train,
                epochs=50,
                batch_size=256,
                shuffle=True,
                validation_data=(x_test_noisy, x_test))

In [ ]:
denoised_images = autoencoder.predict(x_test_noisy)

In [ ]:
n = 3
plt.figure(figsize=(12, 12))

for i in range(n):
    ax = plt.subplot(3, n, i + 1)
    plt.imshow(x_test[i+10].reshape(28, 28), cmap="viridis")
    plt.title("ORIGINAL")
    plt.axis("off")
    ax = plt.subplot(3, n, i + 1 + n)
    plt.imshow(x_test_noisy[i+10].reshape(28, 28), cmap="viridis")
    plt.title("NOISE")
    plt.axis("off")
    ax = plt.subplot(3, n, i + 1 + 2 * n)
    plt.imshow(denoised_images[i+10].reshape(28, 28), cmap='viridis')
    plt.title("DENOISED")
    plt.axis("off")
plt.show()